# Approximation of BEM matrix from vortex rings dynamics problem

For given triangulization of surface $\Omega$ compute $\int_\Omega \frac{\sigma(\mathbf{r})}{\mathbf{r}^3}ds$
Since there is discretization of surface, problem is to compute following matrix multiplication $A\sigma,$
where $\sigma$ is a vector of discretized surface densities, and elements of $A$ are following:

$$A_{ij} = \sigma_i \int_{\Omega_j} \vert \mathbf{r}-\mathbf{r}_i \vert^{-3} ds,$$

here $\Omega_j$ is $j$-th element of surface discretization, $\mathbf{r}_i$ is $i$-th collocation point, $\mathbf{r}$ is a radius vector to element of surface of $\Omega_j$

In [1]:
# main imports and variables

# make this script work in both Python 2 and Python 3
from __future__ import print_function, absolute_import, division

# import all necessary modules
import numpy as np
# at first, get predefined functions and classes from h2py.collections.triangular_surface
from h2tools.collections import triangular_surface as trisurf
# load Problem and ClusterTree classes
from h2tools.problem import Problem, ClusterTree
# import mcbh approximator
from h2tools.mcbh import mcbh

# set parameters
# fname is a filename of surface discretization,
# block_size is a maximum size of leaf nodes of cluster trees,
# tau is an accuracy parameter for MCBH,
# iters is number of iterations for MCBH,
# verbose show if factorization function will be verbose
fname = 'Geodat.dat'
block_size = 25
tau = 1e-5
iters = 1
onfly = 0
verbose = 1

func = trisurf.integral_inverse_r3

In [2]:
# read triangular surface from a file
data = trisurf.TriangularSurface.from_dat(fname)
print('Number of surface discrete elements:', data.count)

Number of surface discrete elements: 4908


In [3]:
# create cluster tree
tree = ClusterTree(data, block_size)
# create main problem object
problem = Problem(func, tree, tree, symmetric=0, verbose=verbose)

Cluster trees are generated in 0.304753065109 seconds
Depth level of each cluster tree: 12
Row cluster tree
    nodes : 573
    leaves: 287
Column cluster tree
    nodes : 573
    leaves: 287


In [4]:
# compute matrix approximation with given parameters
matrix = mcbh(problem, tau=tau, iters=iters, onfly=onfly, verbose=verbose)

# check approximation error
print(matrix.diffnorm(far_only=1))

# Compress matrix
matrix.svdcompress(1e-4, verbose=1)
print(matrix.diffnorm(far_only=1))

matrix.svdcompress(1e-3, verbose=1)
print(matrix.diffnorm(far_only=1))

Far-field interactions(MCBH method):
    Function calls: 8030
    Function values computed: 26239798
    Function time, seconds: 4.19
    Average time per function value, seconds: 1.60e-07
    Maxvol time, seconds: 5.03311252594
Near-field interactions:
    Function calls: 2657
    Function values computed: 1124346
    Function time, seconds: 0.21
    Average time per function value, seconds: 1.88e-07
Total time, seconds: 9.61
Memory:
    Basises, MB: 0.33
    Transfer matrices, MB: 8.29
    Far-field interactions, MB: 24.19
    Near-field interactions, MB: 9.29
Total memory, MB: 42.10
7.35394575014e-05
memory BEFORE SVD-compression: 42.10MB
memory AFTER SVD-compression: 16.50MB
recompression time: 1.27259302139
9.90676569293e-05
memory BEFORE SVD-compression: 16.50MB
memory AFTER SVD-compression: 13.76MB
recompression time: 1.03930211067
0.0010284557832


In [5]:
print(matrix)

H2matrix at 0x103ab7490
    Structure (h2/mcbh): h2
    Memory layout (low/full): full
    Shape: [4908, 4908]
    Total memory, MB: 13.76
        Basises, MB: 0.01
        Transfer matrices, MB: 3.29
        Far-field interactions, MB: 1.17
        Near-field interactions, MB: 9.29
